# Part 2: Semantic Kernel Fundamentals (2025 Edition)

This notebook demonstrates Semantic Kernel's core capabilities with 2025 production patterns. We'll explore the stable plugin architecture, modern kernel orchestration, Agent Framework 1.0, and AutoGen integration that showcase SK's enterprise-ready "Orchestrate Everything" philosophy.

## Environment Setup

First, let's load our environment variables and configure our Azure OpenAI connection for Semantic Kernel.

In [1]:
# Environment and configuration setup
import os
import warnings
import asyncio
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

class SemanticKernelConfig:
    """Configuration management for Azure OpenAI in Semantic Kernel"""
    
    def __init__(self):
        # Load environment variables
        env_path = find_dotenv()
        if env_path:
            load_dotenv(env_path)
            print(f"✅ Loaded environment from: {env_path}")
        else:
            warnings.warn("No .env file found. Using system environment variables only.")
        
        self._load_azure_config()
        self._validate_config()
    
    def _load_azure_config(self):
        """Load Azure OpenAI configuration from environment variables"""
        self.azure_api_key = os.getenv('AZURE_OPENAI_API_KEY')
        self.azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
        self.azure_deployment = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME', 'gpt-4o-mini')
        self.azure_api_version = os.getenv('AZURE_OPENAI_API_VERSION', '2024-10-21')
        self.service_id = "azure_chat"
    
    def _validate_config(self):
        """Validate critical configuration"""
        errors = []
        
        if not self.azure_api_key:
            errors.append("AZURE_OPENAI_API_KEY is required")
        
        if not self.azure_endpoint:
            errors.append("AZURE_OPENAI_ENDPOINT is required")
        
        if errors:
            raise ValueError(f"Configuration errors: {', '.join(errors)}")
        
        print("✅ All Azure OpenAI configuration validated successfully")
    
    def display_config(self):
        """Display current configuration (hiding secrets)"""
        print("Azure OpenAI Configuration for Semantic Kernel:")
        print(f"  Endpoint: {self.azure_endpoint}")
        print(f"  Deployment: {self.azure_deployment}")
        print(f"  API Version: {self.azure_api_version}")
        print(f"  Service ID: {self.service_id}")
        print(f"  API Key: {'*' * 20 + self.azure_api_key[-4:] if self.azure_api_key else 'Not set'}")

# Initialize configuration
try:
    sk_config = SemanticKernelConfig()
    sk_config.display_config()
except Exception as e:
    print(f"❌ Failed to load configuration: {e}")
    print("\nPlease ensure you have a .env file with the following variables:")
    print("- AZURE_OPENAI_API_KEY")
    print("- AZURE_OPENAI_ENDPOINT")
    print("- AZURE_OPENAI_CHAT_DEPLOYMENT_NAME (optional, defaults to gpt-4o-mini)")
    raise

✅ Loaded environment from: c:\Users\chrismckee\Documents\GitHub\SemanticKernelLangChainComparison\.env
✅ All Azure OpenAI configuration validated successfully
Azure OpenAI Configuration for Semantic Kernel:
  Endpoint: https://aideveloperaoai.openai.azure.com
  Deployment: gpt-4.1
  API Version: 2025-01-01-preview
  Service ID: azure_chat
  API Key: ********************6c6e


### 2.1 Basic Chat and Kernel Configuration

Semantic Kernel uses a dependency injection pattern with services registered in the kernel. This provides strong typing and better enterprise patterns.

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.contents import ChatHistory

# Initialize the kernel - central orchestrator for all SK operations
kernel = sk.Kernel()

# Configure Azure OpenAI chat completion service with dependency injection
azure_chat_service = AzureChatCompletion(
    service_id=sk_config.service_id,
    deployment_name=sk_config.azure_deployment,
    endpoint=sk_config.azure_endpoint,
    api_key=sk_config.azure_api_key,
    api_version=sk_config.azure_api_version
)

# Register the service with the kernel (dependency injection pattern)
kernel.add_service(azure_chat_service)

print("🔧 Kernel Configuration:")
print(f"  Services registered: {len(kernel.services)}")
print(f"  Chat service ID: {sk_config.service_id}")

async def demonstrate_basic_chat():
    """Demonstrates basic chat functionality with proper settings"""
    
    # Create chat history for conversation context
    history = ChatHistory()
    history.add_system_message("You are a helpful assistant who provides concise, step-by-step answers.")
    history.add_user_message("What is the capital of France?")
    
    # Create execution settings - required for all SK operations
    execution_settings = AzureChatPromptExecutionSettings(
        service_id=sk_config.service_id,
        max_tokens=150,
        temperature=0.7
    )
    
    print("💬 Basic Chat Example:")
    print("User > What is the capital of France?")
    
    # Execute the chat completion
    response = await azure_chat_service.get_chat_message_content(
        chat_history=history, 
        settings=execution_settings
    )
    
    print(f"Assistant > {response.content}")
    
    # Demonstrate conversation continuation
    history.add_assistant_message(response.content)
    history.add_user_message("What language do they speak there?")
    
    print("\nUser > What language do they speak there?")
    
    response2 = await azure_chat_service.get_chat_message_content(
        chat_history=history,
        settings=execution_settings
    )
    
    print(f"Assistant > {response2.content}")

# Execute the demonstration
await demonstrate_basic_chat()

🔧 Kernel Configuration:
  Services registered: 1
  Chat service ID: azure_chat
💬 Basic Chat Example:
User > What is the capital of France?
Assistant > The capital of France is Paris.
\nUser > What language do they speak there?
Assistant > The official language spoken in Paris, France is French.


### 2.2 Plugin Architecture and Sequential Orchestration

Semantic Kernel's plugin system provides a structured approach to building reusable components. Unlike LangChain's composition, SK uses explicit orchestration.

In [ ]:
# Create semantic functions from prompts (SK's equivalent to LangChain chains)
# These are registered as plugins in the kernel
from semantic_kernel.prompt_template import PromptTemplateConfig, InputVariable
from semantic_kernel.functions import KernelArguments

# Plugin 1: Company Naming
company_naming_prompt = """
What is a good name for a company that makes {{$input}}? 
Provide just the company name, nothing else.
"""

naming_config = PromptTemplateConfig(
    template=company_naming_prompt,
    name="generate_name",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The product description", is_required=True),
    ],
    execution_settings=AzureChatPromptExecutionSettings(
        service_id=sk_config.service_id,
        max_tokens=100,
        temperature=0.7
    )
)

naming_function = kernel.add_function(
    function_name="generate_name", 
    plugin_name="MarketingPlugin", 
    prompt_template_config=naming_config,
)

# Plugin 2: Catchphrase Generation  
catchphrase_prompt = """
Write a creative, memorable catchphrase for the following company: {{$input}}
Make it catchy and professional. Provide just the catchphrase, nothing else.
"""

catchphrase_config = PromptTemplateConfig(
    template=catchphrase_prompt,
    name="generate_catchphrase",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The company name", is_required=True),
    ],
    execution_settings=AzureChatPromptExecutionSettings(
        service_id=sk_config.service_id,
        max_tokens=100,
        temperature=0.8
    )
)

catchphrase_function = kernel.add_function(
    function_name="generate_catchphrase", 
    plugin_name="MarketingPlugin", 
    prompt_template_config=catchphrase_config,
)

print("🔌 Plugins Registered:")
print(f"  Total plugins: {len(kernel.plugins)}")
for plugin_name, plugin in kernel.plugins.items():
    print(f"  - {plugin_name}: {len(plugin.functions)} functions")
    for func_name in plugin.functions:
        print(f"    • {func_name}")

async def demonstrate_orchestration():
    """Demonstrates explicit orchestration - SK's approach to sequential processing"""
    
    product = "colorful, eco-friendly socks"
    print(f"\\n🎯 Input product: {product}")
    print("\\n🔗 Running Sequential Orchestration...")
    
    # Step 1: Generate company name (explicit orchestration)
    print("\\n📝 Step 1: Generating company name...")
    try:
        name_result = await kernel.invoke(naming_function, KernelArguments(input=product))
        company_name = str(name_result).strip()
        print(f"   Result: {company_name}")
    except Exception as e:
        print(f"   Error: {e}")
        return
    
    # Step 2: Generate catchphrase using the generated name
    print("\\n🎨 Step 2: Generating catchphrase...")
    try:
        catchphrase_result = await kernel.invoke(catchphrase_function, KernelArguments(input=company_name))
        catchphrase = str(catchphrase_result).strip()
        print(f"   Result: {catchphrase}")
    except Exception as e:
        print(f"   Error: {e}")
        return
    
    # Display final results
    print("\\n" + "="*50)
    print("✅ ORCHESTRATION COMPLETE")
    print("="*50)
    print(f"Product: {product}")
    print(f"Company Name: {company_name}")
    print(f"Catchphrase: {catchphrase}")
    print("="*50)
    
    return {
        "product": product,
        "company_name": company_name, 
        "catchphrase": catchphrase
    }

# Execute the orchestration
result = await demonstrate_orchestration()

# Demonstrate plugin reusability
print("\\n🔄 Demonstrating Plugin Reusability...")
test_products = ["smart home devices", "artisanal coffee beans"]

for product in test_products:
    print(f"\\n→ Testing with: {product}")
    name_result = await kernel.invoke(naming_function, KernelArguments(input=product))
    print(f"  Generated name: {name_result}")

### 2.3 Native Function Plugins with @kernel_function Decorator

**What this does:** Creates Python classes with the `@kernel_function` decorator to combine traditional code with AI capabilities, demonstrating Semantic Kernel's hybrid AI + code approach.

The code below shows how to build structured, discoverable plugins:
- **Plugin classes** using Python classes with decorated methods for reusable components
- **Function metadata** with descriptions and type annotations for AI discoverability  
- **Parameter descriptions** using `Annotated` types to help the AI understand function inputs
- **Plugin registration** adding multiple plugin instances to the kernel's function registry
- **Direct invocation** calling native functions with proper argument handling

**Native Plugin Benefits:**
- 🔧 **Hybrid AI + Code** - Seamlessly blend LLM capabilities with traditional programming
- 📖 **Self-documenting** - Rich metadata helps AI understand capabilities and usage
- 🎯 **Type safety** - Python type hints provide compile-time validation
- 🔄 **Reusable** - Plugin classes can be instantiated and shared across kernels

**Azure Alternative:** Azure Functions provide similar serverless compute capabilities, though Semantic Kernel's native plugins offer tighter AI integration and automatic discoverability.

In [ ]:
import random
from semantic_kernel.functions import kernel_function, KernelArguments
from datetime import datetime, timedelta
import json
from typing import Annotated

# Native function plugins demonstrate Semantic Kernel's hybrid AI + code approach
# These are Python classes with @kernel_function decorated methods

class MathPlugin:
    """Plugin demonstrating mathematical operations with rich metadata"""
    
    @kernel_function(
        description="Generate a random number within specified bounds for simulations",
        name="GenerateRandomNumber"
    )
    def generate_random_number(
        self, 
        min_val: Annotated[int, "Minimum value (inclusive)"] = 1, 
        max_val: Annotated[int, "Maximum value (inclusive)"] = 100
    ) -> Annotated[str, "Random number as string"]:
        """Generate a random number between specified bounds with error handling"""
        try:
            result = random.randint(min_val, max_val)
            return str(result)
        except ValueError as e:
            return f"Error: Invalid range - {e}"
    
    @kernel_function(
        description="Calculate percentage for analytics and reporting",
        name="CalculatePercentage"
    )
    def calculate_percentage(
        self, 
        part: Annotated[float, "Part value (numerator)"], 
        whole: Annotated[float, "Whole value (denominator)"]
    ) -> Annotated[str, "Percentage formatted as string with % symbol"]:
        """Calculate percentage with proper error handling for division by zero"""
        try:
            if whole == 0:
                return "Error: Cannot divide by zero"
            percentage = (part / whole) * 100
            return f"{percentage:.2f}%"
        except Exception as e:
            return f"Error in calculation: {e}"

class TimePlugin:
    """Plugin for date and time operations with business logic"""
    
    @kernel_function(
        description="Get current date and time in standard format",
        name="GetCurrentDateTime"
    )
    def get_current_datetime(self) -> Annotated[str, "Current date and time in YYYY-MM-DD HH:MM:SS format"]:
        """Returns current date and time in standardized format"""
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    @kernel_function(
        description="Calculate future or past dates by adding days (use negative for past)",
        name="AddDaysToDate"
    )
    def add_days_to_date(
        self, 
        days: Annotated[int, "Number of days to add (negative for past dates)"]
    ) -> Annotated[str, "Future/past date in YYYY-MM-DD format"]:
        """Calculate future or past dates with robust error handling"""
        try:
            target_date = datetime.now() + timedelta(days=days)
            return target_date.strftime("%Y-%m-%d")
        except (OverflowError, ValueError) as e:
            return f"Error: Invalid date calculation - {e}"

class DataPlugin:
    """Plugin for data processing with enterprise-ready functions"""
    
    @kernel_function(
        description="Extract specific field from JSON data with validation",
        name="ExtractJsonField"
    )
    def extract_json_field(
        self, 
        json_data: Annotated[str, "JSON string to parse"], 
        field_name: Annotated[str, "Field name to extract from JSON"]
    ) -> Annotated[str, "Extracted field value or error message"]:
        """Extract field from JSON with comprehensive error handling"""
        try:
            data = json.loads(json_data)
            if field_name in data:
                return str(data[field_name])
            else:
                available_fields = ", ".join(data.keys())
                return f"Field '{field_name}' not found. Available fields: {available_fields}"
        except json.JSONDecodeError as e:
            return f"Error: Invalid JSON format - {e}"
        except Exception as e:
            return f"Error processing JSON: {e}"
    
    @kernel_function(
        description="Count words in text for content analysis",
        name="CountWords"
    )
    def count_words(
        self, 
        text: Annotated[str, "Text content to analyze"]
    ) -> Annotated[str, "Word count as string"]:
        """Count words with whitespace handling and validation"""
        try:
            if not text or not text.strip():
                return "0"
            word_count = len(text.split())
            return str(word_count)
        except Exception as e:
            return f"Error counting words: {e}"

# Register plugins with the kernel - demonstrating plugin discovery
print("🔧 Registering Native Function Plugins...")

math_plugin = kernel.add_plugin(MathPlugin(), "MathPlugin")
time_plugin = kernel.add_plugin(TimePlugin(), "TimePlugin") 
data_plugin = kernel.add_plugin(DataPlugin(), "DataPlugin")

print(f"✅ Successfully registered {len(kernel.plugins)} plugins")

# Display plugin metadata for AI discoverability
print("\n📋 Plugin Registry (AI can discover these capabilities):")
for plugin_name, plugin in kernel.plugins.items():
    print(f"\n  📦 {plugin_name}:")
    for func_name, func in plugin.functions.items():
        print(f"    • {func_name}: {func.description}")
        # Show parameter details for complex functions
        if hasattr(func, 'metadata') and func.metadata.parameters:
            for param in func.metadata.parameters:
                print(f"      - {param.name}: {param.description}")

# Demonstrate direct native function invocation
async def demonstrate_native_plugin_architecture():
    """Showcase native plugin capabilities and metadata-driven AI interaction"""
    
    print("\n🚀 Testing Native Plugin Architecture:")
    print("=" * 60)
    
    # Test MathPlugin with error handling
    print("\n🧮 MathPlugin Capabilities:")
    try:
        # Valid range
        random_result = await kernel.invoke(
            math_plugin["GenerateRandomNumber"], 
            KernelArguments(min_val=10, max_val=50)
        )
        print(f"  ✅ Random number (10-50): {random_result}")
        
        # Percentage calculation
        percentage_result = await kernel.invoke(
            math_plugin["CalculatePercentage"],
            KernelArguments(part=75, whole=200)
        )
        print(f"  ✅ 75 out of 200: {percentage_result}")
        
        # Error case - invalid range
        error_result = await kernel.invoke(
            math_plugin["GenerateRandomNumber"],
            KernelArguments(min_val=100, max_val=50)
        )
        print(f"  ⚠️ Error handling: {error_result}")
        
    except Exception as e:
        print(f"  ❌ Plugin error: {e}")
    
    # Test TimePlugin
    print("\n📅 TimePlugin Capabilities:")
    try:
        current_time = await kernel.invoke(time_plugin["GetCurrentDateTime"])
        print(f"  ✅ Current time: {current_time}")
        
        future_date = await kernel.invoke(
            time_plugin["AddDaysToDate"],
            KernelArguments(days=14)
        )
        print(f"  ✅ Date in 14 days: {future_date}")
        
        past_date = await kernel.invoke(
            time_plugin["AddDaysToDate"],
            KernelArguments(days=-30)
        )
        print(f"  ✅ Date 30 days ago: {past_date}")
        
    except Exception as e:
        print(f"  ❌ Plugin error: {e}")
    
    # Test DataPlugin with real-world scenarios
    print("\n🔍 DataPlugin Capabilities:")
    try:
        # Valid JSON extraction
        sample_json = '{"name": "Alice Smith", "age": 32, "department": "Engineering", "city": "Seattle"}'
        
        name_result = await kernel.invoke(
            data_plugin["ExtractJsonField"],
            KernelArguments(json_data=sample_json, field_name="department")
        )
        print(f"  ✅ Extracted department: {name_result}")
        
        # Missing field case
        missing_result = await kernel.invoke(
            data_plugin["ExtractJsonField"],
            KernelArguments(json_data=sample_json, field_name="salary")
        )
        print(f"  ⚠️ Missing field: {missing_result}")
        
        # Word count analysis
        content = "Semantic Kernel provides a powerful plugin architecture for enterprise applications"
        word_count = await kernel.invoke(
            data_plugin["CountWords"],
            KernelArguments(text=content)
        )
        print(f"  ✅ Word count: {word_count} words")
        
    except Exception as e:
        print(f"  ❌ Plugin error: {e}")
    
    print("\n" + "=" * 60)
    print("✅ Native Plugin Architecture Demonstration Complete")
    print("💡 Key Benefits:")
    print("   • Rich metadata enables AI function discovery")
    print("   • Type annotations provide safety and clarity") 
    print("   • Error handling ensures robust operation")
    print("   • Reusable plugin classes promote modularity")

# Execute the native plugin demonstration
await demonstrate_native_plugin_architecture()

### 2.4 Advanced Function Calling with FunctionChoiceBehavior

**What this does:** Demonstrates Semantic Kernel's advanced function calling capabilities using `FunctionChoiceBehavior` to control exactly how and when the AI model selects and invokes functions.

The code below shows three distinct function calling strategies:
- **AUTO mode** - AI automatically chooses from available functions based on context
- **REQUIRED mode** - Forces AI to call specific functions, ensuring tool usage
- **NONE mode** - Disables function calling while keeping functions visible to the model
- **Selective function control** - Explicitly defining which functions are available for each behavior
- **Advanced configuration** - Concurrent invocation and parallel calling options

**Function Choice Benefits:**
- 🎯 **Precise control** - Choose exactly which functions the AI can access
- ⚡ **Performance optimization** - Concurrent and parallel function execution  
- 🛡️ **Safety management** - Prevent unintended function calls in sensitive contexts
- 📊 **Behavior prediction** - Reliable function calling patterns for production use

**Azure Alternative:** Azure AI services support function calling, though Semantic Kernel's FunctionChoiceBehavior provides more granular control and advanced configuration options for enterprise scenarios.

In [ ]:
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehaviorOptions

# Advanced Function Calling demonstrates precise control over AI function selection
# This showcases the three FunctionChoiceBehavior modes: AUTO, REQUIRED, NONE

async def demonstrate_advanced_function_calling():
    """Showcase FunctionChoiceBehavior patterns for production AI applications"""
    
    print("🎯 Advanced Function Calling with FunctionChoiceBehavior")
    print("=" * 70)
    
    # Scenario 1: AUTO - AI chooses functions automatically based on context
    print("\n🤖 Scenario 1: AUTO Function Choice Behavior")
    print("   • AI automatically selects appropriate functions")
    print("   • Most flexible mode for general-purpose applications")
    
    auto_settings = AzureChatPromptExecutionSettings(
        service_id=sk_config.service_id,
        function_choice_behavior=FunctionChoiceBehavior.Auto(),
        max_tokens=200,
        temperature=0.3
    )
    
    # Create chat history for function calling
    auto_history = ChatHistory()
    auto_history.add_system_message(
        "You are a helpful assistant with access to math, time, and data processing tools. "
        "Use the appropriate functions when needed to answer user questions accurately."
    )
    auto_history.add_user_message(
        "What's the current date and time? Also, what's 25% of 400?"
    )
    
    try:
        print("   🔄 Processing with AUTO behavior...")
        auto_response = await azure_chat_service.get_chat_message_content(
            chat_history=auto_history,
            settings=auto_settings,
            kernel=kernel
        )
        print(f"   ✅ AUTO Response: {auto_response.content}")
        
        # Check if functions were called
        if hasattr(auto_response, 'items') and auto_response.items:
            function_calls = [item for item in auto_response.items if hasattr(item, 'function_name')]
            if function_calls:
                print(f"   🔧 Functions called: {[f.function_name for f in function_calls]}")
        
    except Exception as e:
        print(f"   ❌ AUTO mode error: {e}")
    
    # Scenario 2: REQUIRED - Force AI to use specific functions
    print("\n🎯 Scenario 2: REQUIRED Function Choice Behavior")
    print("   • Forces AI to call specified functions")
    print("   • Ensures tool usage for critical operations")
    
    # Specify which functions must be used
    required_functions = [
        time_plugin["GetCurrentDateTime"],
        math_plugin["CalculatePercentage"]
    ]
    
    required_settings = AzureChatPromptExecutionSettings(
        service_id=sk_config.service_id,
        function_choice_behavior=FunctionChoiceBehavior.Required(
            functions=required_functions,
            options=FunctionChoiceBehaviorOptions(
                allow_concurrent_invocation=True  # Enable concurrent execution
            )
        ),
        max_tokens=200,
        temperature=0.3
    )
    
    required_history = ChatHistory()
    required_history.add_system_message(
        "You must use the available functions to provide accurate information. "
        "Always call the required functions to answer user questions."
    )
    required_history.add_user_message(
        "I need to know what time it is and calculate 40% of 250."
    )
    
    try:
        print("   🔄 Processing with REQUIRED behavior...")
        required_response = await azure_chat_service.get_chat_message_content(
            chat_history=required_history,
            settings=required_settings,
            kernel=kernel
        )
        print(f"   ✅ REQUIRED Response: {required_response.content}")
        print("   💡 Notice: AI was forced to use the specified functions")
        
    except Exception as e:
        print(f"   ❌ REQUIRED mode error: {e}")
    
    # Scenario 3: NONE - Disable function calling
    print("\n🚫 Scenario 3: NONE Function Choice Behavior")
    print("   • Disables all function calling")
    print("   • AI knows about functions but cannot invoke them")
    
    none_settings = AzureChatPromptExecutionSettings(
        service_id=sk_config.service_id,
        function_choice_behavior=FunctionChoiceBehavior.None(),
        max_tokens=200,
        temperature=0.7
    )
    
    none_history = ChatHistory()
    none_history.add_system_message(
        "You are a helpful assistant. You can see available functions but cannot call them. "
        "Explain what you would do if you could use the functions."
    )
    none_history.add_user_message(
        "What's today's date and what's 30% of 150? Show me how you would solve this."
    )
    
    try:
        print("   🔄 Processing with NONE behavior...")
        none_response = await azure_chat_service.get_chat_message_content(
            chat_history=none_history,
            settings=none_settings,
            kernel=kernel
        )
        print(f"   ✅ NONE Response: {none_response.content}")
        print("   💡 Notice: AI explains what it would do but cannot call functions")
        
    except Exception as e:
        print(f"   ❌ NONE mode error: {e}")
    
    # Scenario 4: Selective Function Control
    print("\n🎛️ Scenario 4: Selective Function Control")
    print("   • Choose specific functions for specific contexts")
    print("   • Fine-grained control over available capabilities")
    
    # Only allow time-related functions for this scenario
    time_only_functions = [
        time_plugin["GetCurrentDateTime"],
        time_plugin["AddDaysToDate"]
    ]
    
    selective_settings = AzureChatPromptExecutionSettings(
        service_id=sk_config.service_id,
        function_choice_behavior=FunctionChoiceBehavior.Auto(
            functions=time_only_functions,
            options=FunctionChoiceBehaviorOptions(
                allow_concurrent_invocation=False,  # Sequential execution
                allow_parallel_calls=False  # One function at a time
            )
        ),
        max_tokens=200,
        temperature=0.5
    )
    
    selective_history = ChatHistory()
    selective_history.add_system_message(
        "You are a scheduling assistant with access to time-related functions only. "
        "Help users with date and time calculations."
    )
    selective_history.add_user_message(
        "What's today's date? Also, what date will it be in 15 days? "
        "And can you calculate 25% of 100?"  # This should not trigger math functions
    )
    
    try:
        print("   🔄 Processing with selective function control...")
        selective_response = await azure_chat_service.get_chat_message_content(
            chat_history=selective_history,
            settings=selective_settings,
            kernel=kernel
        )
        print(f"   ✅ Selective Response: {selective_response.content}")
        print("   💡 Notice: Only time functions available, math request handled differently")
        
    except Exception as e:
        print(f"   ❌ Selective mode error: {e}")
    
    # Advanced Configuration Demo
    print("\n⚙️ Advanced Configuration Options:")
    print("   🔧 Concurrent Invocation: Execute multiple functions simultaneously")
    print("   📊 Parallel Calls: Allow AI to select multiple functions in one request")
    print("   🎯 Function Lists: Specify exactly which functions are available")
    print("   🛡️ Safety Controls: Prevent unauthorized function access")
    
    print("\n" + "=" * 70)
    print("✅ Advanced Function Calling Demonstration Complete")
    print("\n💡 Key Takeaways:")
    print("   • AUTO: Maximum flexibility, AI chooses best functions")
    print("   • REQUIRED: Guaranteed function usage, predictable behavior")
    print("   • NONE: Information only, no function execution")
    print("   • Selective: Precise control over available capabilities")
    print("   • Options: Fine-tune execution behavior for production needs")

# Execute the advanced function calling demonstration
await demonstrate_advanced_function_calling()

## Summary: Semantic Kernel Fundamentals

This notebook covered Semantic Kernel's core concepts using enterprise-ready patterns:

### Key Concepts Learned:
1. **Kernel Architecture**: Central orchestrator with dependency injection and service registration
2. **Plugin System**: Structured approach combining semantic functions (AI) and native functions (code)
3. **Explicit Orchestration**: Programmatic control flow rather than compositional chaining
4. **Function Calling**: Automatic function selection with typed interfaces and safety

### Semantic Kernel's Philosophy:
- **"Orchestrate Everything"**: Structured orchestration with clear separation of concerns
- **Hybrid AI + Code**: Seamless integration of LLM capabilities with traditional programming
- **Enterprise Patterns**: Dependency injection, typed interfaces, and governance-ready architecture
- **Plugin Ecosystem**: Reusable, discoverable functions with proper metadata

### SK vs LangChain Comparison:
| Aspect | LangChain (LCEL) | Semantic Kernel |
|--------|------------------|-----------------|
| **Composition** | Pipe operators (`\|`) | Explicit `kernel.invoke()` calls |
| **Functions** | Tools with dynamic binding | Plugins with typed interfaces |
| **Flow Control** | Implicit through chains | Explicit orchestration code |
| **Enterprise Focus** | Community-driven | Microsoft enterprise patterns |

### Next Steps:
- **Multi-Agent Systems**: See `4_semantic_kernel_agents.ipynb` for Agent Framework
- **Advanced Plugins**: Memory integration, planning, and complex workflows
- **Production Deployment**: Azure integration, monitoring, and governance

### Production Considerations:
- Leverage dependency injection for testability and maintainability
- Use typed interfaces for better IDE support and error prevention
- Implement proper error handling and logging throughout the kernel
- Consider plugin security and validation for production deployments
- Take advantage of Azure ecosystem integration for enterprise features

In [ ]:
# This cell is being removed - duplicate content

## 🚀 2025 Agent Framework 1.0 Preview

Semantic Kernel's Agent Framework is moving to GA in Q1 2025. Here's a preview of the stable production patterns.

In [ ]:
# 2025 PATTERN: Agent Framework 1.0 stable API patterns
try:
    from semantic_kernel.agents import ChatCompletionAgent
    from semantic_kernel.agents.group_chat import AgentGroupChat
    from semantic_kernel.contents import ChatMessageContent, AuthorRole
    
    print("✅ Agent Framework 1.0 available (preview mode)")
    
    # Create agents with stable 1.0 API patterns
    research_agent = ChatCompletionAgent(
        service_id=sk_config.service_id,
        kernel=kernel,
        name="ResearchSpecialist",
        instructions="You are a research specialist who provides detailed, well-sourced information on any topic."
    )
    
    analysis_agent = ChatCompletionAgent(
        service_id=sk_config.service_id,
        kernel=kernel,
        name="DataAnalyst", 
        instructions="You are a data analyst who interprets research findings and provides actionable insights."
    )
    
    # Create agent group chat for collaboration
    agent_chat = AgentGroupChat(
        agents=[research_agent, analysis_agent],
        termination_strategy=None  # Will run until completion
    )
    
    print(f"📊 Agent Framework Status:")
    print(f"   Agents created: {len(agent_chat.agents)}")
    print(f"   Research Agent: {research_agent.name}")
    print(f"   Analysis Agent: {analysis_agent.name}")
    
    # Demonstrate agent collaboration
    async def demonstrate_agent_collaboration():
        """Shows how agents collaborate using the 2025 stable API"""
        
        user_message = ChatMessageContent(
            role=AuthorRole.USER,
            content="Research the benefits of renewable energy and provide an analysis of adoption trends."
        )
        
        print(f"\n🎯 User Request: {user_message.content}")
        print("\n🤖 Agent Collaboration in Progress...")
        
        # Add user message to chat
        await agent_chat.add_chat_message(user_message)
        
        # Execute agent collaboration
        async for message in agent_chat.invoke():
            print(f"\n[{message.author.name}]: {message.content}")
            
            # Limit output for demo
            if len(agent_chat.history) >= 4:  # User + 2 agent responses
                break
    
    # Run the collaboration demo
    await demonstrate_agent_collaboration()
    
except ImportError as e:
    print(f"⚠️ Agent Framework not available in current version: {e}")
    print("📅 Agent Framework 1.0 GA expected Q1 2025")
    print("💡 Install preview: pip install semantic-kernel --pre")

## 🔗 2025 AutoGen Integration

Microsoft is providing three convergence approaches for AutoGen + Semantic Kernel integration in early 2025.

In [ ]:
# 2025 PATTERN: AutoGen integration with Semantic Kernel
try:
    # Install: pip install semantic-kernel[autogen]
    from semantic_kernel.agents.autogen import AutoGenAgent
    
    print("✅ AutoGen integration available")
    
    # Create AutoGen conversable agent within SK ecosystem
    autogen_research_agent = AutoGenAgent(
        kernel=kernel,
        name="AutoGenResearcher",
        system_message="You are an advanced research agent with deep analytical capabilities.",
        llm_config={
            'model': sk_config.azure_deployment,
            'temperature': 0.7,
            'azure_endpoint': sk_config.azure_endpoint,
            'api_key': sk_config.azure_api_key
        }
    )
    
    print(f"🔬 AutoGen Agent Created:")
    print(f"   Name: {autogen_research_agent.name}")
    print(f"   Type: AutoGen Conversable Agent in SK Runtime")
    
    # Demonstrate seamless integration
    async def demonstrate_autogen_integration():
        """Shows AutoGen agents working within SK ecosystem"""
        
        # Use AutoGen agent in SK Agent Framework
        mixed_chat = AgentGroupChat(
            agents=[autogen_research_agent, analysis_agent],  # Mix of AutoGen + SK agents
            termination_strategy=None
        )
        
        user_message = ChatMessageContent(
            role=AuthorRole.USER,
            content="Analyze the future of AI agent frameworks and provide strategic recommendations."
        )
        
        print(f"\n🎯 Mixed Agent Team Task: {user_message.content}")
        print("\n🤝 AutoGen + SK Agent Collaboration...")
        
        await mixed_chat.add_chat_message(user_message)
        
        # Show first few exchanges
        response_count = 0
        async for message in mixed_chat.invoke():
            print(f"\n[{message.author.name}]: {message.content[:200]}...")
            response_count += 1
            if response_count >= 2:  # Limit for demo
                break
    
    # Run the mixed integration demo
    await demonstrate_autogen_integration()
    
except ImportError as e:
    print(f"⚠️ AutoGen integration not available: {e}")
    print("📅 AutoGen integration available early 2025")
    print("💡 Install when available: pip install semantic-kernel[autogen]")
except Exception as e:
    print(f"⚠️ AutoGen integration error: {e}")
    print("📝 Note: AutoGen integration is in active development")

## ⚡ 2025 Process Framework Preview

Process Framework GA is expected Q2 2025 for stateful workflows and business process automation.

In [ ]:
# 2025 PATTERN: Process Framework preview for business workflows
try:
    from semantic_kernel.processes import Process, ProcessStep
    
    print("✅ Process Framework preview available")
    
    # Define workflow steps
    class DocumentAnalysisStep(ProcessStep):
        async def execute(self, context):
            """Analyze document content"""
            print(f"📄 Analyzing document: {context.input}")
            # Simulate document analysis
            return f"Analysis complete for {context.input}"
    
    class QualityReviewStep(ProcessStep):
        async def execute(self, context):
            """Quality review step"""
            print(f"🔍 Quality review: {context.data}")
            # Simulate human-in-the-loop review
            return "Quality review passed"
    
    # Create stateful process
    document_process = Process(
        steps=[
            DocumentAnalysisStep(),
            QualityReviewStep(),
        ],
        orchestration_mode='sequential'
    )
    
    print(f"🔄 Process Framework Demo:")
    print(f"   Steps defined: {len(document_process.steps)}")
    print(f"   Orchestration: {document_process.orchestration_mode}")
    
    # Execute process workflow
    async def demonstrate_process_workflow():
        """Shows stateful business process execution"""
        
        input_document = "Technical Requirements Document v2.1"
        print(f"\n🎯 Processing: {input_document}")
        
        result = await document_process.run(input_document)
        print(f"✅ Process completed: {result}")
    
    # Run the process demo
    await demonstrate_process_workflow()
    
except ImportError as e:
    print(f"⚠️ Process Framework not available: {e}")
    print("📅 Process Framework GA expected Q2 2025")
    print("💡 Install preview: pip install semantic-kernel --pre")
except Exception as e:
    print(f"⚠️ Process Framework error: {e}")
    print("📝 Note: Process Framework is in active development")

## Summary: Semantic Kernel Fundamentals (2025 Edition)

This notebook covered the core concepts of Semantic Kernel with 2025 production patterns:

### Key Concepts Learned:
1. **Plugin Architecture**: Structured, reusable components with explicit orchestration
2. **Kernel Orchestration**: Central dependency injection and service management
3. **Function Calling**: Automatic function invocation with structured data flows
4. **Agent Framework 1.0**: Production-ready multi-agent collaboration (Q1 2025 GA)
5. **AutoGen Integration**: Seamless transition from experimentation to production
6. **Process Framework**: Stateful workflows and business process automation (Q2 2025 GA)

### 2025 Semantic Kernel Evolution:
- **Agent Framework 1.0**: Stable API for production multi-agent systems
- **AutoGen Convergence**: Three integration approaches for enhanced capabilities
- **Process Framework**: Enterprise workflow automation with human-in-the-loop
- **Production Readiness**: Enterprise governance and compliance features
- **Azure Integration**: Enhanced cloud-native capabilities

### Next Steps:
- **Advanced Agents**: See `4_semantic_kernel_agents.ipynb` for complex agent patterns
- **Production Deployment**: Enterprise governance and monitoring setup
- **AutoGen Migration**: Transition strategies from pure AutoGen to SK
- **Process Automation**: Business workflow implementation patterns

### 2025 Production Considerations:
- Prepare for Agent Framework 1.0 GA in Q1 2025
- Evaluate AutoGen integration options for advanced agent capabilities
- Plan Process Framework adoption for workflow automation in Q2 2025
- Implement enterprise governance and compliance patterns
- Leverage Azure integration for cloud-native deployment